In [3]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire

Prepare
Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [6]:
df= acquire.get_zillow_data()
df.head()

,Unnamed: 0,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,parcelid.1,logerror,transactiondate,buildingclasstypeid.1,buildingclassdesc,storytypeid.1,storydesc,typeconstructiontypeid.1,typeconstructiondesc,parcelid.2
0,0,2874473,14172985,1.00,7.00,nan,2.50,4.00,nan,nan,...,14172985,0.01,2017-01-03,nan,nan,nan,nan,6.00,Frame,14172985
1,1,803551,14633780,11.00,8.00,nan,1.00,2.00,nan,nan,...,14633780,0.07,2017-01-04,nan,nan,nan,nan,6.00,Frame,14633780
2,2,1325329,14531476,11.00,7.00,nan,2.00,2.00,nan,nan,...,14531476,0.02,2017-01-04,nan,nan,nan,nan,6.00,Frame,14531476
3,3,699288,14668372,11.00,21.00,nan,2.00,2.00,nan,nan,...,14668372,0.14,2017-01-06,nan,nan,nan,nan,6.00,Frame,14668372
4,4,757768,14287849,1.00,7.00,nan,2.00,3.00,nan,nan,...,14287849,-0.03,2017-01-09,nan,nan,nan,nan,6.00,Frame,14287849


In [7]:
df= df.replace([0,' ','NULL'],np.nan)

In [10]:
df=df.dropna(thresh=df.shape[0]*0.6,how='all',axis=1)

In [32]:
df.propertylandusedesc.value_counts()

Condominium                  86
Single Family Residential    33
Name: propertylandusedesc, dtype: int64

In [46]:
def clean_zillow_data():
    '''This function obtains the data using a query connection to sql. Then it drops the columns with more that 60% nulls and drops duplicate columns.'''
    df= acquire.get_zillow_data()
    df= df.replace([0,' ','NULL'],np.nan)
    df= df.dropna(thresh=df.shape[0]*0.6,how='all',axis=1)
    #df= df.drop(df.propertylandusedesc=='condominiums')
    df= df.drop(columns=['Unnamed: 0', 'airconditioningtypeid', 'heatingorsystemtypeid','architecturalstyletypeid', 'bathroomcnt','propertylandusetypeid.1','heatingorsystemtypeid.1', 'architecturalstyletypeid.1', 'airconditioningtypeid.1', 'parcelid.1','typeconstructiontypeid.1', 'id.1', 'parcelid.2' ])
    return df

In [49]:
df2= clean_zillow_data()
df2.head()

,id,parcelid,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,latitude,...,landtaxvaluedollarcnt,taxamount,censustractandblock,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,propertylandusedesc,logerror,transactiondate,typeconstructiondesc
0,2874473,14172985,4.00,2.50,"1,658.00","1,658.00","6,059.00",2.00,2.00,"33,905,686.00",...,"73,035.00","2,795.12","60,590,117,091,010.00",Central,Contemporary,Forced air,Single Family Residential,0.01,2017-01-03,Frame
1,803551,14633780,2.00,1.00,838.00,838.00,"6,059.00",1.00,1.00,"33,739,203.00",...,"79,591.00","1,766.48","60,590,992,482,034.00",Wall Unit,Conventional,Forced air,Condominium,0.07,2017-01-04,Frame
2,1325329,14531476,2.00,2.00,"1,369.00","1,369.00","6,059.00",2.00,1.00,"33,728,400.00",...,"121,715.00","2,551.74","60,590,755,152,008.00",Wall Unit,Contemporary,Radiant,Single Family Residential,0.02,2017-01-04,Frame
3,699288,14668372,2.00,2.00,"1,042.00","1,042.00","6,059.00",2.00,1.00,"33,830,900.00",...,"103,734.00","2,200.12","60,590,870,023,002.00",Wall Unit,Ranch/Rambler,Radiant,Condominium,0.14,2017-01-06,Frame
4,757768,14287849,3.00,2.00,"1,234.00","1,234.00","6,059.00",2.00,1.00,"33,725,799.00",...,"280,991.00","3,969.70","60,590,525,023,011.00",Central,Contemporary,Forced air,Single Family Residential,-0.03,2017-01-09,Frame


Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
A dataframe
A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
The output:
The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
Look up the dropna documentation.
You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
Make use of inplace, i.e. inplace=True/False.

Decide how to handle the remaining missing values:

Fill with constant value.
Impute with mean, median, mode.
Drop row/column